### 프로젝트 실습

## A. 데이터 전처리

### A-1. 타입 통합 & 불량 데이터 처리

#### A-1-1. {데이터타입 통합}

kopo_channel_seasonality_new.csv 파일을 불러온 후 selloutData 변수에 담으세요.
이후 QTY 컬럼 → 실수 (float), 이외컬럼 → 문자(str)로 변경하세요

In [1]:
### 라이브러리 선언
import pandas as pd

In [2]:
### 파일 불러오기
selloutData = \
pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0
...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0
124654,A10,PRODUCT56,201630,275.0
124655,A10,PRODUCT61,201630,0.0
124656,A10,PRODUCT12,201630,15021.0


In [4]:
### 데이터 타입 확인
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [5]:
### 데이터 타입 변경
selloutData = selloutData.astype({"REGIONID":str, "PRODUCT":str, "YEARWEEK":str, "QTY":float})

In [23]:
### 데이터 타입 변경 검증
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
QTY_NEW     float64
YEAR         object
WEEK         object
dtype: object

####  A-1-2. {불량 데이터 처리}

kopo_channel_seasonality_new.csv
자료를 담은
selloutData
변수에서
QTY
컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [7]:
### 불량 데이터 처리 방법 3가지
## 1. np.where 조건 사용 
import numpy as np

selloutData["QTY_NEW"] = \
np.where(selloutData.QTY < 0,
         0,
         selloutData.QTY)

In [8]:
## 2. loc 사용
selloutData.loc[selloutData.QTY < 0, "QTY_NEW"] = 0
selloutData.loc[selloutData.QTY >= 0, "QTY_NEW"] = selloutData.QTY

In [9]:
## 3. 함수 사용
def CleanQTY(inQTY) : 
    # inQTY = 3.4
    outQTY = 0

    if inQTY < 0 :
        outQTY = 0
    else :
        outQTY = inQTY
    return outQTY

In [10]:
selloutData["QTY_NEW"] = \
    selloutData.QTY.apply(CleanQTY)

In [22]:
### 데이터 검증
selloutData.loc[(selloutData.QTY < 0) & (selloutData.QTY_NEW != 0)].count()

REGIONID    0
PRODUCT     0
YEARWEEK    0
QTY         0
QTY_NEW     0
YEAR        0
WEEK        0
dtype: int64

### A-2. 데이터 통합

selloutData
자료에서
YEAR, WEEK
컬럼을 생성하고 WEEK 가 52 이하인
데이터만
조회한 후 refinedSelloutData 변수에 담으세요

In [8]:
### YEAR, WEEK 컬럼 생성하기
selloutData["YEAR"] = selloutData.YEARWEEK.str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:]

In [15]:
### WEEK 컬럼을 str로 비교...? (가능한건지 모르겠음... 값은 같음)
refinedSelloutData = selloutData.loc[selloutData.WEEK <= "52"]

In [13]:
### WEEK 컬럼 int로 변환하여 조건 걸기
refinedSelloutData = selloutData.loc[selloutData.WEEK.astype(int) <= 52]

In [16]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [24]:
### 데이터 검증
refinedSelloutData.loc[selloutData.WEEK.astype(int) > 52].count()

REGIONID    0
PRODUCT     0
YEARWEEK    0
QTY         0
QTY_NEW     0
YEAR        0
WEEK        0
dtype: int64